<a href="https://colab.research.google.com/github/andrePankraz/qa_service/blob/main/notebooks/Neustarthilfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neustarthilfe 2022 FAQ-Suche
Install necessary packages.

In [1]:
!pip install --quiet openai stanza tiktoken sentence-transformers

Import text data:
*   Fetch URL web page with FAQ text and parse HTML
*   Extract FAQ text data into data struct (list of lists)
*   Export as JSON lines file 'faq.jsonl' and as text file 'faq.txt'

In [2]:
import requests
from bs4 import BeautifulSoup, NavigableString
import re
import json

# Convert HTML element into raw inner text:
# Preserve some structure like paragraphs, lists etc.
def process_element(element, indent=''):
    if isinstance(element, NavigableString):
        return element # preserve <span>&nbsp;</span>

    if element.name == 'p':
        content = ''.join(process_element(child, indent) for child in element.children)
        return f'\n{indent}{content.strip()}\n'

    if element.name == 'br':
        return f'\n{indent}'

    if element.name in ['ul', 'ol']:
        items = ['\n']
        for item in element.find_all('li', recursive=False):
            item_content = ''.join(process_element(child, indent + '    ') for child in item.children)
            items.append(f'{indent}- {item_content.strip()}')
        return '\n'.join(items)

    if element.name == 'sup':
        return ''

    # Process other elements and concatenate their content
    content = []
    for child in element.children:
        content.append(process_element(child, indent))
    return ''.join(content)

# Convert FAQ texts from source HTML into a FAQ data struct (list of lists)
def extract_elterngeld_digital_faq():
    # Fetch HTML with FAQ texts:
    url = 'https://www.ueberbrueckungshilfe-unternehmen.de/DE/FAQ/Nsh-22/neustarthilfe-2022.html'
    response = requests.get(url)
    content = response.content.decode('utf-8')

    # Remove potential carriage returns
    content = content.replace('\r', '')
    # Replace newline characters with a space (we have no <pre>)
    content = content.replace('\n', ' ')
    # Replace multiple spaces with a single space
    content = re.sub(r'\s+', ' ', content)

    # Parse HTML
    soup = BeautifulSoup(content, 'html.parser')

    # Target data structure
    faq = [['ID', 'Thema_ID', 'Thema', 'Frage_ID', 'Frage', 'Antwort']]

    # Get relevant root element for FAQ texts
    main_div_element = soup.find('div', class_='accordion__content')

    # Extract FAQ texts into data structure
    group_id = 0
    question_id = 0

    for child in main_div_element.children:
      if child.name == 'h2' and child.get('class') == ['accordion__headline']:
        group_id += 1
        question_group = child.text.strip()
      elif child.name == 'div' and child.get('class') == ['accordion__element']:
        question_id += 1
        question = child.find('h3', class_='accordion__title').text.strip()
        answer_element = child.find('div', class_='accordion__panel')
        answer = process_element(answer_element)
        # Replace multiple "empty lines" (lines with just spaces and \n) with a single newline character
        # and replace "trailing spaces followed by \n" with just "\n"
        answer = re.sub(r'([ ]*\n)+', '\n', answer).strip()
        faq.append([question_id, group_id, question_group, question_id, question, answer])

    return faq

# Export FAQ data struct as JSON lines file 'faq.jsonl'
def write_faq_json(faq):
    with open('faq.jsonl', 'w', encoding="utf-8") as f:
      for entry in faq:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')

# Export FAQ data struct as raw text file (for debugging)
def write_faq_text(faq):
    with open('faq.txt', 'w', encoding="utf-8") as f:
      for item in faq[1:]:
        f.write(f"{item[4]}\n{item[5]}\n")
        f.write('\n')

# Call this functions
faq = extract_elterngeld_digital_faq()
write_faq_json(faq)
write_faq_text(faq)

# Print migrated data for debugging
if False:
    for item in faq:
      print(f"{item[4]}\n{item[5]}\n")

Import JSON lines file 'faq.jsonl' and convert into generic format for document question answering.

In [3]:
def import_faq_documents() -> list[tuple[str, str, str]]:
    faq = []
    with open("faq.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            faq.append(obj)

    # ID, Title, Text
    documents = [(f[0], f[4], f[5]) for f in faq[1:]]
    return documents

documents = import_faq_documents()
# documents

# Tests with Open Source Models (On-Prem)
Load embedding model.

In [4]:
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# OKish models for german (and sometimes also english)...
# German only, quite good and small:
# embedding_model_id = 'PM-AI/bi-encoder_msmarco_bert-base_german'
# Bilingual, quite good and small
# See good Benchmarks: https://huggingface.co/PM-AI/sts_paraphrase_xlm-roberta-base_de-en
# embedding_model_id = 'PM-AI/sts_paraphrase_xlm-roberta-base_de-en' # prefers cos
# German only, OKish and midsized:
# embedding_model_id = 'Sahajtomar/German-semantic'
# Multilingual, pretty good and small
# MPNet seems to be better than RoBERTa variantes, even though very short 128er windows
# embedding_model_id = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
# Multilingual, pretty good and small
# Fine tuned from model before and with longer sequence length of 512
embedding_model_id = 'LLukas22/paraphrase-multilingual-mpnet-base-v2-embedding-all'

embedding_model = SentenceTransformer(embedding_model_id, device=device)
embedding_max_seq_length = embedding_model.max_seq_length

embedding_model.device, embedding_model, embedding_max_seq_length

(device(type='cpu'),
 SentenceTransformer(
   (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
 ),
 512)

Load NLP models for sentence splitting.

In [5]:
import stanza

# configure stanza for sentence splitting in multiple languages
nlp = stanza.MultilingualPipeline(
    lang_id_config={"langid_clean_text": True},
    lang_configs={'de': {'processors': 'tokenize,mwt', 'verbose': False}, 'en': {'processors': 'tokenize', 'verbose': False}})
test = nlp('Initialisiere Deutsche Modelle. Das ist ein Test').sentences

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: multilingual ():
| Processor | Package |
-----------------------
| langid    | ud      |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: langid
INFO:stanza:Done loading processors!


Split documents into paragraphs (chunks / windows) for embedding:
*   Embedding models have a max token size, use it for splitting
*   Split at sentence ends, not in middle of sentences
*   Overlap chunks if possible

In [6]:
import tiktoken

# configure tiktoken for token splitting in target embedding model
embedding_tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002")

def split_into_paragraphs(documents, max_sentences, overlap_sentences, max_tokens, overlap_tokens):
    paragraphs = []  # resulting list

    for nr, document in enumerate(documents):
        id, title, text = document

        # pre-calculate token number for document title (part of embedding paragraph)
        title_tokens = len(embedding_tokenizer.encode(title))

        # split document text into sentences
        nlp_sentences = [s.text for s in nlp(text).sentences]
        # pre-calculate token numbers for each document sentence
        sentence_tokens = [len(embedding_tokenizer.encode(nlp_sentence)) for nlp_sentence in nlp_sentences]

        sentence_index = -1
        paragraph = ''
        paragraph_sentences = 0
        paragraph_tokens = 0

        index = 0
        while index < len(sentence_tokens):
            tokens = sentence_tokens[index]

            if sentence_index == -1:
                # start new paragraph
                sentence_index = index
                paragraph = nlp_sentences[index]
                paragraph_sentences = 1
                paragraph_tokens = title_tokens + 1 + tokens
                index += 1
                continue

            if (max_sentences <= 0 or paragraph_sentences < max_sentences) and (max_tokens <= 0 or paragraph_tokens + tokens <= max_tokens):
                # continue paragraph
                paragraph += ' ' + nlp_sentences[index]
                paragraph_sentences += 1
                paragraph_tokens += 1 + tokens
                index += 1
                continue

            # finish paragraph
            paragraphs.append((nr, sentence_index, title, paragraph, paragraph_tokens))

            # overlap paragraphs with sentence or token window - whatever boundary triggered first
            if max_sentences > 0 and paragraph_sentences == max_sentences and overlap_sentences <= max_sentences / 2:
              index -= overlap_sentences
            if max_tokens > 0 and paragraph_tokens + tokens > max_tokens and overlap_tokens > 0 and overlap_tokens <= max_tokens / 2:
                overlap_tokens_sum = 0
                while index > sentence_index + 1:
                    overlap_tokens_sum += sentence_tokens[index - 1]
                    if overlap_tokens_sum > overlap_tokens:
                        break
                    index -= 1

            # trigger new paragraph
            sentence_index = -1
        else:
            if sentence_index != -1:
                # add final paragraph
                paragraphs.append((nr, sentence_index, title, paragraph, paragraph_tokens))
    return paragraphs


max_sentences = 6
overlap_sentences = 1
max_tokens = embedding_max_seq_length
overlap_tokens = max_tokens / 6

paragraphs = split_into_paragraphs(documents, max_sentences, overlap_sentences, max_tokens, overlap_tokens)

print(f"Splitted {len(documents)} documents into {len(paragraphs)} paragraphs with max sequence length {max_tokens}.")
# paragraphs

Splitted 56 documents into 169 paragraphs with max sequence length 512.


Create embeddings for fact paragraphs.

Prefix the paragraphs with the title, if title isn't already included into the paragraph.

In [7]:
embedding_paragraphs = [p[3] if p[2] in p[3] else p[2] + ': ' + p[3] for p in paragraphs]

embeddings = embedding_model.encode(embedding_paragraphs)

print(f"Embedded {len(embeddings)} paragraphs with {embeddings.shape[1]} dimensions each.")

Embedded 169 paragraphs with 768 dimensions each.


Create embedding for question.

In [8]:
question = 'Welche Regelungen gelten für die Berechnung des Elterngelds bei Frühgeborene?'

query_embedding = embedding_model.encode(question)

print(f"Embedded 1 question with {len(query_embedding)} dimensions.")

Embedded 1 question with 768 dimensions.


Find best embeddings via [k-nearest-neighbors (kNN)](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) with [Cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity).

In [9]:
import numpy as np

# L2-normalize -> dot-score is than same like cosine-similarity
embeddings = embeddings / np.sqrt((embeddings**2).sum(1, keepdims=True))
query_embedding = query_embedding / np.sqrt((query_embedding**2).sum())

similarities = embeddings.dot(query_embedding)
top_similarities = np.argsort(-similarities)

print('Top 30 paragraphs:')
for k in top_similarities[:30]:
    print(f"(Score: {similarities[k]:.4f})  {paragraphs[k]}")

Top 30 paragraphs:
(Score: 0.7356)  (46, 0, '6.3 Welche Regelungen gelten, wenn ich 2019 in Elternzeit war und daher geringere oder keine Umsätze aus selbständiger Tätigkeit hatte?', 'Berechnung des Referenzumsatzes bei Elternzeit in 2019\nFür Antragstellende, die im Jahr 2019 Elternzeit in Anspruch genommen haben, besteht stets die Möglichkeit die Elternzeit als Unterbrechung der Geschäftstätigkeit (= außergewöhnlicher Umstand) zu behandeln und den Referenzumsatz nach Punkt 6.2 berechnen zu lassen. Auf Anforderung der Bewilligungsstellen sind entsprechende Nachweise bereitzustellen. Berechnung des Referenzumsatzes bei vollständiger Elternzeit im Jahr 2019\nAntragstellende, die 2019 vollständig in Elternzeit waren, können sich auch entscheiden, alternativ den Referenzumsatz für 2019 auf Basis des Elterngeldes zu ermitteln. Als (dreimonatiger) Referenzumsatz gilt dann 25 Prozent des im Jahr 2019 erhaltenen Elterngeldes zuzüglich eines 15-prozentigen Aufschlages auf das in 2019 erhaltene

Recluster found paragraphs with same title.

In [10]:
def longest_overlap_suffix_prefix(s1, s2):
    for i in range(len(s1)-1, -1, -1):
        if s2.startswith(s1[i:]):
            return s1[i:]
    return ""

def merge_overlapping_fragments(fragments):
    result = fragments[0]
    for i in range(1, len(fragments)):
        overlap = longest_overlap_suffix_prefix(result, fragments[i])
        if len(overlap) == 0:
            result += ' ' + fragments[i]
        else:
            result += fragments[i][len(overlap):]
    return result

def restruct_top_paragraphs(nested_list):
    clustered_items = {}

    for item in nested_list:
        if item[0] not in clustered_items:
            clustered_items[item[0]] = []
        clustered_items[item[0]].append(item)

    # Sort each cluster by the second entry
    for key in clustered_items:
        clustered_items[key] = sorted(clustered_items[key], key=lambda x: x[1])

    # Combine clusters in the original order
    reordered_list = []
    for key in clustered_items:
        reordered_list.append([key, clustered_items[key][0][2], merge_overlapping_fragments([item[3] for item in clustered_items[key]])])

    return reordered_list

top_paragraphs = [paragraphs[k] for k in top_similarities[:30]]
top_paragraphs = restruct_top_paragraphs(top_paragraphs)

# top_paragraphs

Aggregate facts into a text corpus:
*   Use references that can be parsed out of generated response [[x]]
*   Restrict text corpus to max token size (function argument)

In [11]:
import tiktoken

prompt_tokenizer = tiktoken.encoding_for_model("gpt-4")

def top_facts(top_paragraphs, max_facts_tokens):
    facts = ''
    facts_tokens = 0

    for top_paragraph in top_paragraphs:
        fact = '[[' + str(top_paragraph[0]) + ']] ' + top_paragraph[1] + ' ' + top_paragraph[2]
        fact_tokens = len(prompt_tokenizer.encode(fact))
        if facts_tokens + fact_tokens > max_facts_tokens:
            break
        facts += '\n' + fact
        facts_tokens += 1 + fact_tokens

    return facts

facts = top_facts(top_paragraphs, 2500)

print(facts)
print(f"Tokens: {len(prompt_tokenizer.encode(facts))}")


[[46]] 6.3 Welche Regelungen gelten, wenn ich 2019 in Elternzeit war und daher geringere oder keine Umsätze aus selbständiger Tätigkeit hatte? Berechnung des Referenzumsatzes bei Elternzeit in 2019
Für Antragstellende, die im Jahr 2019 Elternzeit in Anspruch genommen haben, besteht stets die Möglichkeit die Elternzeit als Unterbrechung der Geschäftstätigkeit (= außergewöhnlicher Umstand) zu behandeln und den Referenzumsatz nach Punkt 6.2 berechnen zu lassen. Auf Anforderung der Bewilligungsstellen sind entsprechende Nachweise bereitzustellen. Berechnung des Referenzumsatzes bei vollständiger Elternzeit im Jahr 2019
Antragstellende, die 2019 vollständig in Elternzeit waren, können sich auch entscheiden, alternativ den Referenzumsatz für 2019 auf Basis des Elterngeldes zu ermitteln. Als (dreimonatiger) Referenzumsatz gilt dann 25 Prozent des im Jahr 2019 erhaltenen Elterngeldes zuzüglich eines 15-prozentigen Aufschlages auf das in 2019 erhaltene Elterngeld (Referenzumsatz = 40 Prozent d

Load an extracting Question Answering (QA) model.

In [12]:
import torch
from transformers import pipeline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

qa_model_id = 'timpal0l/mdeberta-v3-base-squad2'
qa_pipeline = pipeline('question-answering', model=qa_model_id, tokenizer=qa_model_id, device=device)

qa_pipeline.device

device(type='cuda', index=0)

Find answers with an extracting Question Answering (QA) model. These QA models find the most relevant text fragments for the given question (as citation or reference) and cannot really do reasoning or create answers.

The texts in this FAQ are too complicated for such a small model. We need a much bigger model!

In [13]:
qa_pipeline({
    'question': question,
    'context': facts
}, top_k=10)

[{'score': 0.0002434718335280195,
  'start': 1946,
  'end': 2028,
  'answer': ' Löhne und Gehälter aus sozialversicherungspflichtigen Beschäftigungsverhältnissen'},
 {'score': 4.392413029563613e-05,
  'start': 6731,
  'end': 6745,
  'answer': ' nachzuweisen.'},
 {'score': 3.444353205850348e-05,
  'start': 6692,
  'end': 6745,
  'answer': ' auf Anforderung der Bewilligungsstelle nachzuweisen.'},
 {'score': 3.27426714648027e-05,
  'start': 6678,
  'end': 6745,
  'answer': ' begründen und auf Anforderung der Bewilligungsstelle nachzuweisen.'},
 {'score': 3.0354240152519196e-05,
  'start': 1946,
  'end': 1965,
  'answer': ' Löhne und Gehälter'},
 {'score': 2.445508107484784e-05,
  'start': 7266,
  'end': 7327,
  'answer': ' Darüberhinausgehende Sonderregelungen sind nicht vorgesehen.'},
 {'score': 2.0070958271389827e-05,
  'start': 1746,
  'end': 1780,
  'answer': ' („Einnahmen im Vergleichszeitraum'},
 {'score': 1.4775281670154072e-05,
  'start': 1512,
  'end': 1552,
  'answer': ' Einnahm

# With OpenAI Models
Import Open API key.

In [14]:
from google.colab import drive
import json
import openai

drive.mount('/content/drive')
with open('/content/drive/My Drive/Private/api_keys.json', 'r') as f:
    api_keys = json.load(f)
openai.api_key = api_keys['openai']

Mounted at /content/drive


In [15]:
prompt = """Du bist eine herausragende Suchmaschine, die in einem Webportal natürlichsprachige Fragen beantwortet.
Es folgt eine Frage und ein Kontext mit mehreren Fakten.
Nutze für die Antwort nur die gegebenen Fakten.
Wenn kein Fakt zur Antwort passt, dann antworte mit 'Ich bin mir nicht sicher.'
Ignoriere dabei Fakten, die nicht unmittelbar relevant für die Frage sind (nicht alle Fakten passen zur Frage).
Fokussiere bei der Antwort auf den relevantesten Fakt und bleibe beim Thema (nicht abwschweifen).
Die Fakten beginnen jeweils mit einer Referenzangabe im Format [[x]]. Zitiere zu einem verwendeten Fakt jeweils die entsprechende Referenzangabe [[x]].
Antworte kurz und prägnant in maximal 250 Wörtern. Formuliere in verständlichen Sätzen und nutze einfache Sprache.
Denke Schritt für Schritt und prüfe Deine Antwort.

Frage: """ + question + '\n\nKontext:\n-----' + facts + '\n-----\nAntwort:'
print(prompt)
print(f"Tokens: {len(prompt_tokenizer.encode(prompt))}")

Du bist eine herausragende Suchmaschine, die in einem Webportal natürlichsprachige Fragen beantwortet.
Es folgt eine Frage und ein Kontext mit mehreren Fakten.
Nutze für die Antwort nur die gegebenen Fakten.
Wenn kein Fakt zur Antwort passt, dann antworte mit 'Ich bin mir nicht sicher.'
Ignoriere dabei Fakten, die nicht unmittelbar relevant für die Frage sind (nicht alle Fakten passen zur Frage).
Fokussiere bei der Antwort auf den relevantesten Fakt und bleibe beim Thema (nicht abwschweifen).
Die Fakten beginnen jeweils mit einer Referenzangabe im Format [[x]]. Zitiere zu einem verwendeten Fakt jeweils die entsprechende Referenzangabe [[x]].
Antworte kurz und prägnant in maximal 250 Wörtern. Formuliere in verständlichen Sätzen und nutze einfache Sprache.
Denke Schritt für Schritt und prüfe Deine Antwort.

Frage: Welche Regelungen gelten für die Berechnung des Elterngelds bei Frühgeborene?

Kontext:
-----
[[46]] 6.3 Welche Regelungen gelten, wenn ich 2019 in Elternzeit war und daher ger

Call generative language model.

In [16]:
response = openai.Completion.create(model="text-davinci-003", prompt=prompt, max_tokens=500)
response

<OpenAIObject text_completion id=cmpl-7DXCMZsM565Z6Sy5pYYKBYVjNtbZ7 at 0x7f94c6989030> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Der Referenzumsatz des Elterngelds bei Fr\u00fchgeborene wird nach [[46]] ermittelt. Hierzu kann entweder der Referenzmonatsumsatz aus dem Jahr 2019, der auf 25 Prozent des im Jahr 2019 erhaltenen Elterngeldes zuz\u00fcglich eines 15-Prozentigen Aufschlages auf das im Jahr 2019 erhaltenen Elterngelds (Referenzumsatz = 40 Prozent des Elterngeldes 2019) berechnet wird, oder aber ein alternativer Vergleichsumsatz herangezogen werden. Dieser betr\u00e4gt den durchschnittlichen monatlichen Umsatz aller vollen Monate der selbst\u00e4ndigen/gewerblichen Gesch\u00e4ftst\u00e4tigkeit im Jahr 2019. Sollte der Referenzumsatz nicht nachvollziehbar ausgepr\u00e4gt sein, so kann als Alternative der gesch\u00e4tzter Jahresumsatz 2021 herangezogen werden."
    }
  ],
  "created": 1683460234,
  "id": 

Extract response text.

In [17]:
response['choices'][0]['text'].strip()

'Der Referenzumsatz des Elterngelds bei Frühgeborene wird nach [[46]] ermittelt. Hierzu kann entweder der Referenzmonatsumsatz aus dem Jahr 2019, der auf 25 Prozent des im Jahr 2019 erhaltenen Elterngeldes zuzüglich eines 15-Prozentigen Aufschlages auf das im Jahr 2019 erhaltenen Elterngelds (Referenzumsatz = 40 Prozent des Elterngeldes 2019) berechnet wird, oder aber ein alternativer Vergleichsumsatz herangezogen werden. Dieser beträgt den durchschnittlichen monatlichen Umsatz aller vollen Monate der selbständigen/gewerblichen Geschäftstätigkeit im Jahr 2019. Sollte der Referenzumsatz nicht nachvollziehbar ausgeprägt sein, so kann als Alternative der geschätzter Jahresumsatz 2021 herangezogen werden.'